### Imports

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sns.set(style="whitegrid", context="notebook")

### Load the Data

In [41]:
cohort_path = "../data/3195663216_msprime_sim_cohort.csv"
effects_path = "../data/3195663216_msprime_effect_sizes.csv"

cohort = pd.read_csv(cohort_path)
effects = pd.read_csv(effects_path)

### Identify Numerical + Categorical Features

In [42]:
numeric_features = ["age", "env_index", "polygenic_score", "quant_trait", "disease_prob", "PC1", "PC2"]

categorical_features = ["sex", "disease_status"]

pcs = [col for col in cohort.columns if col.startswith("PC")]

numeric_features, categorical_features

(['age',
  'env_index',
  'polygenic_score',
  'quant_trait',
  'disease_prob',
  'PC1',
  'PC2'],
 ['sex', 'disease_status'])

### Train and Test Split

In [43]:
X = cohort[numeric_features + categorical_features]
y = cohort["quant_trait"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

X_train.shape, X_test.shape

((7000, 9), (3000, 9))

### Standardization

In [44]:
scaler = StandardScaler()

# Fit *only* on training numerical features
X_train_scaled_num = scaler.fit_transform(X_train[numeric_features])
X_test_scaled_num = scaler.transform(X_test[numeric_features])

# Rebuild full DataFrames with same column order
X_train_scaled = pd.DataFrame(
    X_train_scaled_num,
    columns=numeric_features,
    index=X_train.index
)
X_test_scaled = pd.DataFrame(
    X_test_scaled_num,
    columns=numeric_features,
    index=X_test.index
)

# Add categorical features
for col in categorical_features:
    X_train_scaled[col] = X_train[col].values
    X_test_scaled[col] = X_test[col].values

X_train_scaled.head()

,age,env_index,polygenic_score,quant_trait,disease_prob,PC1,PC2,sex,disease_status
9069,-0.951245,-0.385407,-1.745874,-1.658782,-1.697634,-1.666482,-1.081947,0,1
2603,0.416455,0.299589,-0.116855,-0.836093,-0.127086,-1.699625,-0.606496,0,0
7738,0.553225,-0.651264,0.978902,-0.308846,0.342096,2.083987,-0.817477,0,1
1579,1.168690,-1.364159,-0.024322,-0.576059,-0.645353,-0.369216,0.407394,1,1
5058,-0.130625,0.865069,-0.171785,0.621501,0.172615,-0.218088,0.147418,0,0


### Matrices

In [45]:
X_train_baseline = X_train_scaled[["polygenic_score"]]
X_test_baseline = X_test_scaled[["polygenic_score"]]

print(X_train_baseline, X_test_baseline)

      polygenic_score
9069        -1.745874
2603        -0.116855
7738         0.978902
1579        -0.024322
5058        -0.171785
...               ...
5734        -0.352876
5191         0.228400
5390        -0.367079
860          0.287106
7270         1.008366

[7000 rows x 1 columns]       polygenic_score
6252         0.451030
4684         1.955427
1731         1.502939
4742         0.990733
4521        -0.815776
...               ...
8014        -0.273383
1074        -1.040422
3063        -0.918017
6487         0.576913
4705         0.569873

[3000 rows x 1 columns]


In [46]:
core_vars = numeric_features + categorical_features

X_train_full = X_train_scaled[core_vars]
X_test_full = X_test_scaled[core_vars]

In [47]:
if pcs:
    full_pca_vars = core_vars + pcs
    X_train_full_pca = X_train_scaled[full_pca_vars]
    X_test_full_pca = X_test_scaled[full_pca_vars]
else:
    X_train_full_pca = X_train_full.copy()
    X_test_full_pca = X_test_full.copy()

X_train_full_pca.head()

,age,env_index,polygenic_score,quant_trait,disease_prob,PC1,PC2,sex,disease_status,PC1,PC2
9069,-0.951245,-0.385407,-1.745874,-1.658782,-1.697634,-1.666482,-1.081947,0,1,-1.666482,-1.081947
2603,0.416455,0.299589,-0.116855,-0.836093,-0.127086,-1.699625,-0.606496,0,0,-1.699625,-0.606496
7738,0.553225,-0.651264,0.978902,-0.308846,0.342096,2.083987,-0.817477,0,1,2.083987,-0.817477
1579,1.168690,-1.364159,-0.024322,-0.576059,-0.645353,-0.369216,0.407394,1,1,-0.369216,0.407394
5058,-0.130625,0.865069,-0.171785,0.621501,0.172615,-0.218088,0.147418,0,0,-0.218088,0.147418


In [48]:
X_train_scaled[numeric_features].mean(), X_train_scaled[numeric_features].std()

(age               -8.932537e-17
 env_index         -3.045183e-18
 polygenic_score   -1.827110e-17
 quant_trait       -1.395709e-17
 disease_prob       6.191872e-17
 PC1               -2.131628e-17
 PC2                1.167320e-17
 dtype: float64,
 age                1.000071
 env_index          1.000071
 polygenic_score    1.000071
 quant_trait        1.000071
 disease_prob       1.000071
 PC1                1.000071
 PC2                1.000071
 dtype: float64)

The means are almost 0 and the SD are close to 1. No data leakage

In [49]:
X_test_scaled[numeric_features].mean(), X_test_scaled[numeric_features].std()

(age                0.028347
 env_index         -0.000676
 polygenic_score    0.033285
 quant_trait        0.002218
 disease_prob       0.023020
 PC1                0.027729
 PC2               -0.000158
 dtype: float64,
 age                1.002912
 env_index          1.029294
 polygenic_score    0.988079
 quant_trait        0.970384
 disease_prob       0.984546
 PC1                1.016331
 PC2                0.986007
 dtype: float64)

Made a mistake, quant_trait and disease_prob should not be standardized. They should be removed

### Updated

In [50]:
numeric_features = ["age", "env_index", "polygenic_score", "PC1", "PC2"]

categorical_features = ["sex"]

pcs = [col for col in cohort.columns if col.startswith("PC")]

numeric_features, categorical_features

(['age', 'env_index', 'polygenic_score', 'PC1', 'PC2'], ['sex'])

In [51]:
X = cohort[numeric_features + categorical_features]
y = cohort["quant_trait"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

X_train.shape, X_test.shape

((7000, 6), (3000, 6))

In [52]:
scaler = StandardScaler()

# Fit *only* on training numerical features
X_train_scaled_num = scaler.fit_transform(X_train[numeric_features])
X_test_scaled_num = scaler.transform(X_test[numeric_features])

# Rebuild full DataFrames with same column order
X_train_scaled = pd.DataFrame(
    X_train_scaled_num,
    columns=numeric_features,
    index=X_train.index
)
X_test_scaled = pd.DataFrame(
    X_test_scaled_num,
    columns=numeric_features,
    index=X_test.index
)

# Add categorical features (sex)
X_train_scaled["sex"] = X_train["sex"].values
X_test_scaled["sex"] = X_test["sex"].values
X_train_scaled.head()

,age,env_index,polygenic_score,PC1,PC2,sex
9069,-0.951245,-0.385407,-1.745874,-1.666482,-1.081947,0
2603,0.416455,0.299589,-0.116855,-1.699625,-0.606496,0
7738,0.553225,-0.651264,0.978902,2.083987,-0.817477,0
1579,1.168690,-1.364159,-0.024322,-0.369216,0.407394,1
5058,-0.130625,0.865069,-0.171785,-0.218088,0.147418,0


In [53]:
X_train_baseline = X_train_scaled[["polygenic_score"]]
X_test_baseline = X_test_scaled[["polygenic_score"]]

print(X_train_baseline, X_test_baseline)

      polygenic_score
9069        -1.745874
2603        -0.116855
7738         0.978902
1579        -0.024322
5058        -0.171785
...               ...
5734        -0.352876
5191         0.228400
5390        -0.367079
860          0.287106
7270         1.008366

[7000 rows x 1 columns]       polygenic_score
6252         0.451030
4684         1.955427
1731         1.502939
4742         0.990733
4521        -0.815776
...               ...
8014        -0.273383
1074        -1.040422
3063        -0.918017
6487         0.576913
4705         0.569873

[3000 rows x 1 columns]


In [54]:
core_vars = numeric_features + categorical_features

X_train_full = X_train_scaled[core_vars]
X_test_full = X_test_scaled[core_vars]

In [55]:
if pcs:
    full_pca_vars = core_vars + pcs
    X_train_full_pca = X_train_scaled[full_pca_vars]
    X_test_full_pca = X_test_scaled[full_pca_vars]
else:
    X_train_full_pca = X_train_full.copy()
    X_test_full_pca = X_test_full.copy()

X_train_full_pca.head()

,age,env_index,polygenic_score,PC1,PC2,sex,PC1,PC2
9069,-0.951245,-0.385407,-1.745874,-1.666482,-1.081947,0,-1.666482,-1.081947
2603,0.416455,0.299589,-0.116855,-1.699625,-0.606496,0,-1.699625,-0.606496
7738,0.553225,-0.651264,0.978902,2.083987,-0.817477,0,2.083987,-0.817477
1579,1.168690,-1.364159,-0.024322,-0.369216,0.407394,1,-0.369216,0.407394
5058,-0.130625,0.865069,-0.171785,-0.218088,0.147418,0,-0.218088,0.147418


In [56]:
X_train_scaled[numeric_features].mean(), X_train_scaled[numeric_features].std()

(age               -8.932537e-17
 env_index         -3.045183e-18
 polygenic_score   -1.827110e-17
 PC1               -2.131628e-17
 PC2                1.167320e-17
 dtype: float64,
 age                1.000071
 env_index          1.000071
 polygenic_score    1.000071
 PC1                1.000071
 PC2                1.000071
 dtype: float64)

In [57]:
X_test_scaled[numeric_features].mean(), X_test_scaled[numeric_features].std()

(age                0.028347
 env_index         -0.000676
 polygenic_score    0.033285
 PC1                0.027729
 PC2               -0.000158
 dtype: float64,
 age                1.002912
 env_index          1.029294
 polygenic_score    0.988079
 PC1                1.016331
 PC2                0.986007
 dtype: float64)